In [5]:
import pandas as pd

# The Parser module is used to convert the raw data to a standard format
from mobvis.preprocessing.parser import Parser as par

# The Locations module is used to find the Geo-locations of the trace, used by almost all metrics
from mobvis.metrics.utils.Locations import Locations as loc
# The HomeLocations module is used to find the Home-locations of the trace, used by some metrics
from mobvis.metrics.utils.HomeLocations import HomeLocations as hloc
# The Contacts module is used to detect the Contacts between the nodes, used by Social metrics
from mobvis.metrics.utils.Contacts import Contacts as cnt

# The MetricBuilder module can be used to instantiate all the metrics 
from mobvis.metrics.utils.MetricBuilder import MetricBuilder as mb

# The metric and spatial plotters contains all the visualizations of MobVis
from mobvis.plots.metric_plotter import *
from mobvis.plots.spatial_plotter import *
%pip install nbformat

In [6]:
""" --- ADJUSTABLE PARAMETERS --- """
maxdistance = 15 # max distance to use when calculating the stay locations
pause = 1 # time needed (in minutes) for a stay location to become a geo location
disttype = "euclidean" # distance to use when calculating the contacts
communicationRadius = 15 # radius to use when calculating the contacts
traceToAnalyze = 5 # ID of the trace to analyze (script will analyze the ID given in traceNames)

# True corresponds to reading the given file => use when you "cached" the data somewhere 
readFile = {
    "mainTrace": True,
    "parsedTrace": True,
    "stayAndGeoLoc": True,
    "homeLoc": True,
    "RADG": True,
    "TRVD": True,
    "VISO": False,
    "TRVT": True,
    "VIST": True,
    "CONT": True,
    "INCO": True,
    "CONTACTS": True
}

# True corresponds to generating and saving a given file => use when you want to cache the data 
# If you dont have the data cached yet, set the value to True in readFile and toSave
# After you got the data, set the toSave value to False again and leave the readFile value as True
toSave = {
    "parsedTrace": True,
    "stayAndGeoLoc": True,
    "homeLoc": True,
    "RADG": True,
    "TRVD": True,
    "VISO": False,
    "TRVT": True,
    "VIST": True,
    "CONT": True,
    "INCO": True,
    "CONTACTS": True
}

""" --- END OF ADJUSTABLE PARAMETERS --- """

traceDetailsString = "T" + str(traceToAnalyze) + "_dist" + str(maxdistance) + "_pause" + str(pause)
# e.g. T1_dist15_pause1
traceRoot = "./traces/" # root directory, where all the traces are stored

# Add the directory names of your trace here
traceNames = {
    1: "trace1_09-06-2024_03.17.29.711", 
    2: "trace2_09-06-2024_03.17.41.719", 
    3: "trace3_09-06-2024_03.19.12.999",
    4: "aSIOTmm_30-07-2024_18.39.23.577", # 0.9
    5: "aSIOTmm_30-07-2024_18.44.34.013" # 0.1
    }
# tracePath = traceRoot + traceNames[traceToAnalyze] + "/pos_trace5DaysAddedZerosFixed.csv"
tracePath = traceRoot + traceNames[traceToAnalyze] + "/pos_traceAddedZerosAndFixedAndLowsInterpolatedNaNRemoved.csv"

metricsPath = traceRoot + traceNames[traceToAnalyze] + "/metrics/"
cachePath = traceRoot + traceNames[traceToAnalyze] + "/cached/"
#
# fileName = name + "_dist_" + str(maxdistance) + "_pause_" + str(pause)
# if metricsPath and cachePath do not exist, create them
# import os 
import os
if not os.path.exists(metricsPath):
    os.makedirs(metricsPath)
if not os.path.exists(cachePath):
    os.makedirs(cachePath)

In [7]:

trace = pd.read_csv(tracePath, names=['id', 'timestamp', 'x', 'y'], delimiter=',', skiprows=1)


In [ ]:

# id, timestamp, x, y
if toSave["parsedTrace"]:
    parsed_trace = par.parse(raw_trace=trace, is_ordered=True)
    parsed_trace.to_csv(cachePath + "parsedTrace_" + traceDetailsString +  ".csv", header=True, index=False)
if readFile["parsedTrace"]:
    parsed_trace = pd.read_csv(cachePath + "parsedTrace_" + traceDetailsString + ".csv", delimiter=',', names=['id', 'timestamp', 'x', 'y'], skiprows=1)
# copyTrace = parsed_trace.copy() 


### Find Stay Locations and Geo Locations of the trace

In [ ]:
# parsed_trace = pd.read_csv(cachePath + "parsedTrace_" + traceDetailsString + ".csv", delimiter=',', names=['id', 'timestamp', 'x', 'y'], skiprows=1)

if toSave["stayAndGeoLoc"]:
    trace_loc, loc_centers = loc.find_locations(trace=parsed_trace, max_d=maxdistance, pause_threshold=pause, dist_type=disttype)
    trace_loc.to_csv(cachePath + "trace_loc_" + traceDetailsString + ".csv", header=True, index=False)
    loc_centers.to_csv(cachePath + "loc_centers_" + traceDetailsString + ".csv", header=True, index=False)
if readFile["stayAndGeoLoc"]:
    trace_loc = pd.read_csv(cachePath + "trace_loc_" + traceDetailsString + ".csv", delimiter=',', names=['id', 'x', 'y', 'sl', 'gl', 'index', 'timestamp'], skiprows=1)
    loc_centers = pd.read_csv(cachePath + "loc_centers_" + traceDetailsString + ".csv", delimiter=',', names=['id', 'sl', 'x', 'y', 'max_x', 'min_x', 'max_y', 'min_y'], skiprows=1)

trace_loc = trace_loc.astype({"id": int, "x": float, "y": float, "sl": int, "gl": bool, "index": float, "timestamp": float})
loc_centers = loc_centers.astype({"id": int, "sl": int, "x": float, "y": float, "max_x": float, "min_x": float, "max_y": float, "min_y": float})

### Home locations

In [ ]:
if toSave["homeLoc"]:
    trace_homes = hloc.find_homes(trace_loc=trace_loc)
    trace_homes.to_csv(cachePath + "trace_homes" + traceDetailsString + ".csv", header=True, index=False)
if readFile["homeLoc"]:
    trace_homes = pd.read_csv(cachePath + "trace_homes" + traceDetailsString + ".csv", delimiter=',', names=['id', 'home_location', 'x', 'y'], skiprows=1)

# trace_homes.head()
trace_homes = trace_homes.astype({"id": int, "home_location": int, "x": float, "y": float})

## Metrics

## Spatial Metrics (RADG, TravelDistance, VisitOrder)

##### Note: VisitOrder takes a hell lot of time and doesn't really provide much of an insight. You probably want to extract this metric last.

When calculating RADG you need the home locations as an input. When you extract them with MobVis though, they tend to be incorrect. Therefore use the ground truth data from the mobility model as the input.

In [ ]:
# real ground truth home locations given by the mobility model
realHomes = pd.read_csv(traceRoot + traceNames[traceToAnalyze] + "/home_locationsFixed.csv", delimiter=',', names=['id', 'x', 'y'], skiprows=1)

# join realHomes with trace_homes on id
merged = pd.merge(trace_homes, realHomes, on='id', how='inner', suffixes=('_trace', ''))
merged.head()


if toSave["RADG"]:
    trace_radg = mb.build_metric('RADG', trace=parsed_trace, trace_loc=trace_loc, sl_centers=loc_centers, homes=merged, dist_type="Euclidean").extract()
    trace_radg.to_csv(metricsPath + "RADGRealHomes" + traceDetailsString + ".csv", header=True, index=False)
if readFile["RADG"]:
    trace_radg = pd.read_csv(metricsPath + "RADGRealHomes" + traceDetailsString + ".csv", delimiter=',', names = ["id","home_location","radius_of_gyration"], skiprows=1)

trace_radg = trace_radg.astype({"id": int, "home_location": int, "radius_of_gyration": float})



In [ ]:
if toSave["TRVD"]:
    trace_trvd = mb.build_metric('TRVD', trace_loc=trace_loc, dist_type="Euclidean").extract()
    trace_trvd.to_csv(metricsPath + "TRVD" + traceDetailsString + ".csv", header=True, index=False)
if readFile["TRVD"]:
    trace_trvd = pd.read_csv(metricsPath + "TRVD" + traceDetailsString + ".csv", delimiter=',', names = ["id","travel_distance","init_sl","final_sl","ix","iy","fx","fy"], skiprows=1)

trace_trvd = trace_trvd.astype({"id": int, "travel_distance": float, "init_sl": int, "final_sl": int, "ix": float, "iy": float, "fx": float, "fy": float})


VisitOrder has some flaws, therefore you need to do it in this complicated way. 

In [14]:
# if toSave["VISO"]:
# # Calculate Stay and Geo locations again, because VISO only works if GL's have been calculated by loc.geo_locations lmao
# # Reset index of geos because VISO metric cant extract it otherwise lmao
#     sl = loc.stay_locations_euclidean(parsed_trace, maxdistance)
#     geos = loc.geo_locations(sl, pause)
#     geos1 = geos.loc[geos.gl == True]
#     geos1.reset_index(drop=True, inplace=True)
    
#     trace_viso = mb.build_metric('VISO', trace_loc=geos1).extract()
#     trace_viso.to_csv(metricsPath + "VISO" + traceDetailsString + ".csv", header=True, index=False)
# if readFile["VISO"]:
#     trace_viso = pd.read_csv(metricsPath + "VISO" + traceDetailsString + ".csv", delimiter=',', names=["id","timestamp","x","y","sl","visit_order"], skiprows=1)

# trace_viso = trace_viso.astype({"id": int, "timestamp": float, "x": float, "y": float, "sl": int, "visit_order": int})



In [15]:
# # geos1 = geos[geos['gl'] == True]
# geos1 = geos.loc[geos.gl == True]
# geos1.reset_index(drop=True, inplace=True)
# # print(int(geos1.id[0]))
# print(geos1.id)
# if toSave["VISO"]:
#     trace_viso = mb.build_metric('VISO', trace_loc=geos1).extract()
#     trace_viso.to_csv(metricsPath + "VISO" + traceDetailsString + ".csv", header=True, index=False)
# if readFile["VISO"]:
#     trace_viso = pd.read_csv(metricsPath + "VISO" + traceDetailsString + ".csv", delimiter=',', skiprows=1)
# # # # # Add NAMES 

# if saveData:
#     trace_contacts = cnt.detect_contacts(df=parsed_trace, radius=0.13, dist_type="euclidean")
#     trace_contacts.to_csv(fileName + ".trace_contacts.csv", header=True, index=False)

## Temporal Metrics (TravelTime, VisitTime)

In [ ]:
if toSave["TRVT"]:
    trace_trvt = mb.build_metric('TRVT', trace_loc=trace_loc).extract()
    trace_trvt.to_csv(metricsPath + "TRVT" + traceDetailsString + ".csv", header=True, index=False)
if readFile["TRVT"]:
    trace_trvt = pd.read_csv(metricsPath + "TRVT" + traceDetailsString + ".csv", delimiter=',', names=["id","init_sl","final_sl","t_exit","t_arrival","travel_time"], skiprows=1)

trace_trvt = trace_trvt.astype({"id": int, "init_sl": int, "final_sl": int, "t_exit": float, "t_arrival": float, "travel_time": float})



In [ ]:
if toSave["VIST"]:
    trace_vist = mb.build_metric('VIST', trace_loc=trace_loc).extract()
    trace_vist.to_csv(metricsPath + "VIST" + traceDetailsString + ".csv", header=True, index=False)
if readFile["VIST"]:
    trace_vist = pd.read_csv(metricsPath + "VIST" + traceDetailsString + ".csv", delimiter=',', names=["id", "timestamp", "sl", "visit_time"], skiprows=1)

trace_vist = trace_vist.astype({"id": int, "timestamp": float, "sl": int, "visit_time": float})


## Social Metrics

In [ ]:
# Communication range = 50m radius
if toSave["CONT"]:
    trace_contacts = cnt.detect_contacts(df=parsed_trace, radius=communicationRadius, dist_type=disttype)
    trace_contacts.to_csv(metricsPath + "CONT_" + str(communicationRadius) + "_" + traceDetailsString + ".csv", header=True, index=False)
if readFile["CONT"]:
    trace_contacts = pd.read_csv(metricsPath + "CONT_" + str(communicationRadius) + "_" + traceDetailsString + ".csv", delimiter=',', names=["id1", "id2", "x1", "y1", "x2", "y2", "timestamp"], skiprows=1)

trace_contacts = trace_contacts.astype({"id1": int, "id2": int, "x1": float, "y1": float, "x2": float, "y2": float, "timestamp": float})

type(trace_contacts)


In [ ]:
if toSave["INCO"]:
    trace_inco = mb.build_metric('INCO', contacts_df=trace_contacts).extract()
    trace_inco.to_csv(metricsPath + "INCO" + traceDetailsString + ".csv", header=True, index=False)
if readFile["INCO"]:
    trace_inco = pd.read_csv(metricsPath + "INCO" + traceDetailsString + ".csv", delimiter=',', names=["id1", "id2", "intercontact_time"], skiprows=1)

trace_inco = trace_inco.astype({"id1": int, "id2": int, "intercontact_time": float})